
<h1 id="1.데이터-로드">1.데이터 로드<a class="anchor-link" href="#1.데이터-로드">¶</a></h1>


In [ ]:

import pandas as pd
data = pd.read_csv("./word_embedding.csv")



In [ ]:

#csv파일을 데이터프레임으로 바꾸고, 0번째 칼럼 삭제
#data1 : 데이터프레임으로 바꾼 데이터
#data2 : 단어 이름 칼럼을 제거한 데이터
data1 = pd.DataFrame(data)
data2 = data1.drop(["0"], axis=1)



In [ ]:

data2.shape



In [ ]:

#데이터 정규화
#data3 : 정규화한 데이터
from sklearn.preprocessing import scale
data3 = pd.DataFrame(scale(data2))




<h1 id="2.-K-Means">2. K-Means<a class="anchor-link" href="#2.-K-Means">¶</a></h1>


In [ ]:

#K-means로 군집화
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=10, random_state=0).fit(data3)



In [ ]:

#군집화한 라벨을 추가
kmeans.labels_
predict = pd.DataFrame(kmeans.labels_)
predict.columns=['predict']



In [ ]:

#군집화 결과와 데이터를 concatenate
#data4 : 군집화한 라벨을 붙인 데이터(원본데이터+kmeans)
data4 = pd.concat([data3,predict],axis=1)



In [ ]:

data4.head()



In [ ]:

#차원축소 전 데이터로 시각화
import matplotlib.pyplot  as plt

plt.figure(figsize=(12,8))
plt.scatter(data4[0],data4[1],c=data4['predict'], alpha=0.5)
plt.show()




<h1 id="3.-차원축소">3. 차원축소<a class="anchor-link" href="#3.-차원축소">¶</a></h1>



<h3 id="1)-PCA">1) PCA<a class="anchor-link" href="#1)-PCA">¶</a></h3>


In [ ]:

from sklearn.decomposition import PCA
pca = PCA().fit(data3)



In [ ]:

#고유값(주성분의 설명비율)
pca.explained_variance_



In [ ]:

#원래 피쳐의 개수는 100개 > 100개의 고유값이 나옴
len(pca.explained_variance_)



In [ ]:

#scree plot으로 적절한 주성분 수 확인하기
fig = plt.figure(figsize=(8,5))
sing_vals = range(1,data2.shape[1]+1)
eigvals = pca.explained_variance_
plt.plot(sing_vals, eigvals)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')
plt.show()



In [ ]:

#주성분이 10개 이내에서 확 꺾이므로 10개내로 다시 자세히 보기
fig = plt.figure(figsize=(8,5))
sing_vals = range(1,11)
eigvals = pca.explained_variance_[:10]
plt.plot(sing_vals, eigvals)
plt.title('Scree Plot')
plt.xlabel('Principal Component')
plt.ylabel('Eigenvalue')
plt.show()



In [ ]:

#4개정도 선택하는게 적절해보인다. 주성분을 4개로 했을 때 설명비율은?
pca_var = sum(pca.explained_variance_[:4]) / sum(pca.explained_variance_) 
print(pca_var)



In [ ]:

#주성분 4개로 데이터 return
new_pca = PCA(n_components = 4)
pca_features = pd.DataFrame(new_pca.fit_transform(data3)) #차원축소 결과



In [ ]:

#K-means로 군집화(군집수 10개)
pca_kmeans = KMeans(n_clusters=10, random_state=0).fit(pca_features)

#군집화한 라벨을 추가
pca_kmeans.labels_
pca_predict = pd.DataFrame(pca_kmeans.labels_)
pca_predict.columns=['pca_predict']

#군집화 결과와 데이터를 concatenate
#data4 : pca후 군집화한 라벨을 붙인 데이터(pca.transform + kmeans)
data5 = pd.concat([pca_features,pca_predict],axis=1)



In [ ]:

data5.head()



In [ ]:

#차원축소 후 데이터로 시각화
plt.figure(figsize=(12,8))
plt.scatter(data5[0],data5[1],c=data5['pca_predict'],alpha=0.5)
plt.show()




<h2 id="2)-t-SNE">2) t-SNE<a class="anchor-link" href="#2)-t-SNE">¶</a></h2>


In [ ]:

#2개의 성분으로 TSNE적용
from sklearn.manifold import TSNE

tsne = TSNE(n_components=2)
tsne_transformed = tsne.fit_transform(data3)



In [ ]:

#transform 결과가 array -> dataframe으로 변경
tsne_transformed = pd.DataFrame(tsne_transformed)



In [ ]:

#K-means로 군집화
tsne_kmeans = KMeans(n_clusters=10, random_state=0).fit(tsne_transformed)

#군집화한 라벨을 추가
tsne_kmeans.labels_
tsne_predict = pd.DataFrame(tsne_kmeans.labels_)
tsne_predict.columns=['tsne_predict']

#군집화 결과와 데이터를 concatenate
#data6 : tSNE후 군집화한 라벨을 붙인 데이터
data6 = pd.concat([tsne_transformed,tsne_predict],axis=1)



In [ ]:

#차원축소 후 데이터로 시각화
plt.figure(figsize=(12,8))
plt.scatter(data6[0],data6[1],c=data6['tsne_predict'],alpha=0.5)
plt.show()

